In [1]:
# data wrangling imports
import numpy as np
import pandas as pd

# other imports
import csv
import re
import requests

In [2]:
liefe_df = pd.read_csv("../geotracker/data/lieferando_restaurants.csv")
# creating column to identify the data source
liefe_df["database"] = "lieferando"
# dropping duplicated columns
liefe_df.drop_duplicates(inplace=True)

# updating city column, to Berlin
liefe_df.city = "Berlin"

# converting avg review into 1-10 scale
liefe_df.avg_review_score = liefe_df.avg_review_score * 2


In [12]:
# preprocessing type_of_cuisine
liefe_df.type_of_cuisine = liefe_df.type_of_cuisine.apply(
    lambda x: x.split(",")[0])
liefe_df['type_of_cuisine_categorized'] = liefe_df.type_of_cuisine.str.lower()

In [13]:
# dictionary containing all keywords and categorizing
unique_toc_dict = {
    'thai': 'asian',
    'fine-dining': 'middle eastern',
    'cheese': np.nan,
    'fusion': np.nan,
    'butterchicken': 'middle eastern',
    'chinese': 'asian',
    'traditional': np.nan,
    'café': 'cafes',
    'german': 'european',
    'cocktail': 'bars',
    'hamburger': 'american',
    'baklava': 'middle eastern',
    'pokebowl': 'poke',
    'bagel': 'breakfast/dessert',
    'waffles': 'cafes',
    'worklunch': 'fastfood',
    'tapas': 'mediterranean',
    'mediterranean': 'mediterranean',
    'pastries': 'breakfast/dessert',
    'sliders': 'fastfood',
    'turkish': 'middle eastern',
    'steak': 'steak',
    'snacks': 'snacks',
    'Sashimi': 'asian',
    'pancakes': 'breakfast/dessert',
    'Georgian': 'european',
    'risotto': 'italian',
    'glutenfree': 'healthy',
    'bento': 'asian',
    'bistro': 'european',
    'shawarma': 'middle eastern',
    'meatballs': 'european',
    'sushi': 'asian',
    'fish': 'seafood',
    'Don': np.nan,
    'bakery': 'breakfast/dessert',
    'delicious': np.nan,
    'mexican': 'mexican',
    'summerrolls': 'asian',
    'chickennuggets': 'fastfood',
    'korean': 'asian',
    'vegan': 'vegetarian or vegan',
    'chocolate': 'breakfast/dessert',
    'porridge': 'breakfast/dessert',
    'Schnitzel': 'european',
    'Arabic': 'middle eastern',
    'moussaka': 'middle eastern',
    'Dessert': 'breakfast/dessert',
    'vegetarian': 'vegetarian or vegan',
    'donut': 'breakfast/dessert',
    'friedchicken': 'fastfood',
    'smoothie': 'breakfast/dessert',
    'beer': 'bars',
    'indian': 'middle eastern',
    'rice': 'asian',
    'fruit': 'breakfast/dessert',
    'icecoffee': 'cafes',
    'Pastrami': np.nan,
    'dumplings': 'asian',
    'currywurst': 'fastfood',
    'asian': 'asian',
    'pizza': 'italian',
    'grill': 'steak',
    'seafood': 'seafood',
    'wine': 'bars',
    'schnitzel': 'european',
    'american': 'american',
    'healthy': 'healthy',
    'hummus': 'middle eastern',
    'russian': 'russian',
    'Donburi': np.nan,
    'baguette': 'breakfast/dessert',
    'salad': 'healthy',
    'fries': 'fastfood',
    'Austrian': 'european',
    'gyoza': 'asian',
    'potato': np.nan,
    'naan': 'nan',
    'icecream': 'breakfast/dessert',
    'pita': 'mediterranean',
    'sausage': 'european',
    'neapolitanpizza': 'italian',
    'spaghetti': 'italian',
    'cake': 'breakfast/dessert',
    'ribs': 'american',
    'dessert': "breakfast/dessert",
    'Doughnut': 'breakfast/dessert',
    'matcha': 'breakfast/dessert',
    'focaccia': 'italian',
    'homemade': np.nan,
    'milkshake': 'american',
    'taco': 'mexican',
    'curry': 'fastfood',
    'israeli': 'middle eastern',
    'bapburgersandpastrami': '',
    'sandwich': 'fastfood',
    'bowl': 'poke',
    'maki': 'asian',
    'roll': 'asian',
    'brunch': 'breakfast/dessert',
    'pho': 'asian',
    'vietnamese': 'asian',
    'burgers': 'fastfood',
    'muchapizza': 'italian',
    'masala': 'middle eastern',
    'contemporary': np.nan,
    'falafel': 'middle eastern',
    'gyros': 'greek',
    'friedrice': 'asian',
    'chicken': 'snacks',
    'italian': 'european',
    'spaetzle': 'european',
    'streetfood': 'fastfood',
    'Mozzarella': 'italian',
    'ramen': 'asian',
    'antipasti': 'italian',
    'noodles': 'italian',
    'hotdog': 'snacks',
    'coffee': 'cafes',
    'oriental': 'middle eastern',
    'bubbletea': 'breakfast/dessert',
    'greek': 'mediterranean',
    'middleeastern': 'middle eastern',
    'pasta': 'italian',
    'fresh': 'healthy',
    'tandoori': 'middle eastern',
    'wrap': 'healthy',
    'european': 'european',
    'tea': 'breakfast/dessert',
    'bao': 'asian',
    'beyondmeat': 'vegetarian or vegan',
    'duck': 'asian',
    'galette': 'european',
    'panini': 'italian',
    'soup': 'healthy',
    'fastfood': 'fastfood',
    'LatinAmerican': 'south american',
    'juice': 'breakfast/dessert',
    'halal': 'middle eastern',
    'burger': 'american',
    'vegetable': 'vegetarian or vegan',
    'Hawaii': 'american',
    'breakfast': 'breakfast/dessert',
    'poke': 'poke',
    'drinks': 'bars',
    'burrito': 'mexican',
    'salmon': 'seafood',
    'homecooking': np.nan,
    'spanish': 'mediterranean',
    'meat': 'steak',
    'french': 'european',
    'veggieburger': 'vegetarian or vegan',
    'lunch': 'steak',
    'meze': 'asian',
    'homemademeals': np.nan,
    'kebab': 'middle eastern',
    'beef': 'steak',
    'tex-mex': 'mexican',
    'japanese': 'asian',
    'arab' : 'middle eastern',
    'italian style pizza' : 'italian',
    'spanish/tapas' : 'mediterranean',
    'drinks/snacks': 'bars',
    '100% halal':'middle eastern',
    'bio': 'healthy',
    'ice cream' :'breakfast/dessert',
    'steaks':'steak',
    'sandwiches' :'fastfood',
    'turkish pizza' : 'middle eastern',
    'salads': 'healthy',
    'indonesian': 'asian',
    'german dishes':'european',
    'döner': 'middle eastern',
    'argentinian': 'south american',
    'wraps':'healthy',
    'desserts':'breakfast/dessert',
    'spare ribs':'steak',
    'lebanese': 'middle eastern',
    'moroccan':'middle eastern',
    'polish': 'european',
    'gluten-free': "healthy",
    'american style pizza': 'american',
    'austrian cuisine': 'european',
    'soups': 'healthy',
    'african': 'middle eastern',
    'other': np.nan,
    'balkans':'european',
    'iranian': 'middle eastern',
    'baked goods': 'breakfast/dessert',
    '': np.nan
}

# in case we need to add more columns

for x in liefe_df.type_of_cuisine.unique():
    if x not in unique_toc_dict.keys():
        unique_toc_dict[x] = np.nan

#to check if sth's missing
# for x in liefe_df.type_of_cuisine.unique():
#     if x not in unique_toc_dict:
#         print(x)


In [14]:
liefe_df['address'] = liefe_df.street + ", " + liefe_df.zip_code + ", " + liefe_df.city

In [15]:
# adding lat/lon
def latitude(address):
    try:
        params = {"q": address, "format": "json"}
        places = requests.get(f"https://nominatim.openstreetmap.org/search",
                          params=params).json()[0]['lat']
        return places
    except Exception:
        return np.nan

def longitude(address):
    try:
        params = {"q": address, "format": "json"}
        places = requests.get(f"https://nominatim.openstreetmap.org/search",
                              params=params).json()[0]['lon']
        return places
    except Exception:
        return np.nan


In [18]:
liefe_df['latitude'] = liefe_df.address.apply(latitude)
liefe_df['longitude'] = liefe_df.address.apply(longitude)


In [ ]:
# creating clean df
liefe_df_clean = liefe_df[[
    'restaurant_name', 'avg_review_score', 'reviews', 'type_of_cuisine_categorized',
    'street', 'zip_code', 'city_name', 'latitude', 'longitude', 'database']]

KeyError: "['city_name'] not in index"

In [ ]:
# saving de into csv
wolt_df_clean.to_csv("../geotracker/data/wolt_clean_data.csv")

In [ ]:
liefe_df_clean.shape